# Explore Nasdaq-100 ↔ leveraged/inverse ETF relationships
This notebook loads `data/` outputs produced by `etf-mapper refresh` and shows a few basic joins.


In [ ]:
import pandas as pd
from pathlib import Path
import sqlite3

HERE = Path.cwd()

def find_data_dir(start: Path) -> Path | None:
    # Look in start, then parents, for:
    #   data/edges.parquet OR data/universe.sqlite
    for root in [start, *start.parents]:
        d = root / "data"
        if (d / "universe.sqlite").exists() or (d / "edges.parquet").exists():
            return d
    # Also handle: universe.sqlite exists somewhere below
    for root in [start, *start.parents]:
        candidates = list(root.glob("**/universe.sqlite"))
        if candidates:
            return candidates[0].parent
    return None

DATA = find_data_dir(HERE)
if DATA is None:
    raise FileNotFoundError(
        "Could not find a data directory with edges.parquet or universe.sqlite.\n"
        "Generate it first from the repo root:\n\n"
        "  python -m etf_mapper.cli refresh --out data\n\n"
        "Then re-run this notebook."
    )

print("Using data directory:", DATA)

def load_table(name: str) -> pd.DataFrame:
    parquet_path = DATA / f"{name}.parquet"
    sqlite_path = DATA / "universe.sqlite"

    # Prefer parquet if present + engine exists
    if parquet_path.exists():
        try:
            return pd.read_parquet(parquet_path)
        except Exception:
            pass

    if sqlite_path.exists():
        with sqlite3.connect(sqlite_path) as conn:
            return pd.read_sql(f"SELECT * FROM {name}", conn)

    raise FileNotFoundError(f"Missing table '{name}' in {DATA}")

# New canonical tables
try:
    equities = load_table("equities")
except Exception:
    equities = load_table("nasdaq100_constituents")

try:
    etfs = load_table("etfs")
except Exception:
    etfs = load_table("levered_etfs")

edges = load_table("edges")

equities.head()


In [ ]:
edges.head()

In [ ]:
# Basic sanity: Nasdaq-100 should be ~100 constituents
print("Equities:", len(equities), "unique symbols:", equities['symbol'].nunique())

# Derivative exposure coverage by underlying (single-stock + index edges)
const = set(equities["symbol"].astype(str).str.upper())
deriv = edges[edges["edge_type"].eq("derivative_exposure")].copy()
deriv["src"] = deriv["src"].astype(str).str.upper()

mapped = deriv[deriv["src"].isin(const)]
print("Derivative edges for Nasdaq-100 symbols:", len(mapped))
mapped.groupby("src")["dst"].nunique().sort_values(ascending=False).head(25)


In [ ]:
def show_stock(symbol: str, top_n: int = 50) -> pd.DataFrame:
    """Human-readable view of all ETF relationships we currently know for a stock."""
    sym = symbol.upper().strip()
    e = edges[edges["src"].astype(str).str.upper().eq(sym)].copy()
    if e.empty:
        return e

    e = e.merge(etfs.rename(columns={"ticker":"dst"}), on="dst", how="left", suffixes=("","_etf"))

    cols = [
        "src","dst","edge_type",
        "relationship_group","direction","leverage_multiple",
        "strategy_group","relationship","daily_target","issuer","name","source_url","weight","asof"
    ]
    cols = [c for c in cols if c in e.columns]
    e = e[cols].sort_values(["edge_type","relationship_group","direction","leverage_multiple","issuer","dst"])
    return e.head(top_n)

# Example: NVDA / TSLA
show_stock("NVDA")


In [ ]:
# Coverage report: which Nasdaq-100 names have any derivative products today?
coverage = mapped.groupby("src")["dst"].nunique().rename("n_derivative_etfs").reset_index()
coverage = equities[["symbol","company"]].merge(coverage, left_on="symbol", right_on="src", how="left").drop(columns=["src"])
coverage["n_derivative_etfs"] = coverage["n_derivative_etfs"].fillna(0).astype(int)
coverage.sort_values("n_derivative_etfs", ascending=False).head(30)


In [ ]:
# NOTE:
# This project is now structured to support holdings-based 'every ETF that holds NVDA/TSLA/etc' expansion.
# The holdings provider is currently a stub (returns 0 rows), so edges are mostly derivative_exposure.
# Next step is to implement a real holdings provider (SEC N-PORT or a vendor API) and emit edges with:
#   edge_type='holds', weight=<holding %>, asof=<date>.
